In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [2]:
df = pd.read_csv("Data/power-data/save_data/train_process.csv")
df = df.drop(["Unnamed: 0"], axis=1)

In [3]:
df.head()

,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),generator_temperature(°C),...,windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),windmill_generated_power(kW/h),atmospheric_temperature(°K),mat_do_khi_quyen
0,2,2,94.820023,0.371184,41.723019,-0.903423,82.410573,42.523015,2563.124522,76.665560,...,20.767246,239.836388,2730.310605,42.084666,2.217542,0.314065,24.281689,6.766521,174.000000,2.070208
1,3,2,241.832734,27.764785,-99.000000,-9.656611,44.104919,46.258870,2372.384119,78.129803,...,20.767246,337.944723,1780.207200,107.888643,4.210346,0.448494,27.262139,5.966275,300.764785,0.197261
2,1,2,95.484724,0.371184,41.855473,12.652763,42.322098,42.878552,1657.169646,67.654469,...,45.033197,227.850294,1666.049900,-42.931459,2.719475,0.302321,27.366127,2.874342,273.371184,0.205495
3,1,1,238.819424,0.371184,45.443914,15.115323,44.759643,47.282101,2888.134079,95.389974,...,44.827154,492.081520,1964.502895,42.744596,4.857385,0.367140,24.287767,14.851089,174.000000,0.374183
4,6,2,10.722890,0.371184,41.981183,1.715696,-17.616459,43.469852,781.695419,37.423065,...,20.865737,259.274601,1177.516152,13.387289,2.248762,0.453374,27.971650,3.519074,273.371184,1.458688


In [4]:
X = df.drop(['windmill_generated_power(kW/h)'], axis=1)
y = df['windmill_generated_power(kW/h)']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

 Remove the hight correlation

In [6]:
# lam cho thuat toan nhanh hon
# tao khoang trong bit cho cac feature
# tranh overfitting
def correlation(dataset, threshold):
    col_corr = set()
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

In [7]:
corr_feature = correlation(X_train, 0.8)
print("Correlation Feature:", len(corr_feature))

Correlation Feature: 2


In [8]:
corr_feature

{'generator_temperature(°C)', 'mat_do_khi_quyen'}

In [9]:
X_train.drop(labels=corr_feature, axis=1)
X_test.drop(labels=corr_feature, axis=1)

,turbine_status_impute_mode,cloud_level_impute_mode,wind_speed(m/s),atmospheric_temperature(°C),shaft_temperature(°C),blades_angle(°),gearbox_temperature(°C),engine_temperature(°C),motor_torque(N-m),atmospheric_pressure(Pascal),area_temperature(°C),windmill_body_temperature(°C),wind_direction(°),resistance(ohm),rotor_torque(N-m),blade_length(m),blade_breadth(m),windmill_height(m),atmospheric_temperature(°K)
18419,13,1,-121.256687,22.784201,43.266757,-9.656611,11.433379,41.582521,2109.217213,53048.942126,36.990955,44.268855,257.780195,1699.601221,38.045256,2.960598,0.459823,34.679266,295.784201
15909,9,1,32.494780,11.798903,-5.300211,-1.069185,44.917840,41.493475,895.652692,123588.707929,26.594508,41.486682,223.421703,1345.337612,-56.231571,3.641664,0.480725,17.968470,284.798903
18318,11,2,20.594481,11.599239,43.677175,-9.656611,44.879601,43.692938,821.279342,-27428.496469,30.871969,20.865737,239.243219,1281.737701,14.246644,3.104885,0.415148,27.219702,284.599239
23647,13,1,94.101591,18.721167,46.337774,-1.131772,48.049781,43.619410,2752.232203,18361.778724,45.712962,43.511750,0.000000,1869.951080,43.427423,4.370876,0.367207,27.023873,291.721167
7241,9,1,92.024674,20.406279,41.449477,-9.656611,40.692350,40.585019,1083.579721,16441.393334,21.469727,41.227142,44.802554,1665.383448,22.031056,4.686664,0.366413,25.372842,293.406279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3953,13,2,92.897743,20.610433,43.480141,-1.056697,-7.160819,43.495173,2718.173741,53048.942126,40.568612,42.187517,242.304491,1880.030128,42.285202,4.133384,0.480939,26.452663,293.610433
21863,1,1,93.986542,19.321092,90.590594,-1.087198,83.256219,44.635393,2322.134505,16832.782181,37.026331,43.385197,242.210823,1762.120252,37.817020,0.838611,0.433496,26.972639,292.321092
2085,13,1,35.412255,10.289493,45.204084,-9.656611,45.863665,43.991071,918.792265,134750.863833,44.809644,45.799243,474.754001,1362.826380,-36.683819,2.248762,0.371655,48.228843,283.289493
17835,14,1,25.734781,0.371184,66.047879,-1.087183,-130.012016,45.184916,872.085859,408247.481963,33.217468,42.840418,205.049940,1314.577927,15.777533,4.923513,0.393053,26.536670,174.000000


In [24]:
sfs1 = SFS(RandomForestRegressor(), k_features=10, forward=True, floating=False, verbose=2, scoring='r2', cv=3)
sfs1 = sfs1.fit(np.array(X_train), y_train)